In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(10, 5) # 这里的前一项是10，所以自己构造的数据长度也要为10；
        self.fc2 = nn.Linear(5, 10)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [3]:
net = Net()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [4]:
# 设置随机数生成器的种子，保证每次rand结果相同
torch.manual_seed(0) # 不会影响梯度更新；
src = torch.rand(2,10)
src

tensor([[0.4963, 0.7682, 0.0885, 0.1320, 0.3074, 0.6341, 0.4901, 0.8964, 0.4556,
         0.6323],
        [0.3489, 0.4017, 0.0223, 0.1689, 0.2939, 0.5185, 0.6977, 0.8000, 0.1610,
         0.2823]])

In [5]:
tgt = (src>0.5).float() # tgt = (src>0.5).long()用于生成pytorch的整数；
tgt

tensor([[0., 1., 0., 0., 0., 1., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1., 0., 0.]])

In [6]:
# 使用TensorDataset将自定义的输入和标签打包；
dataset = TensorDataset(src, tgt)

# 使用DataLoader将数据打包成batch
batch_size = 2
trainloader = DataLoader(dataset, batch_size=batch_size)

In [7]:
# 训练模型
for epoch in range(100):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data # 每次抽取的data就是一个批次，以当天batch为例，即为2；
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() # loss.item()为当前batch损失，将其相加用于计算整个epoch的平均损失；
    if epoch%10==0: # 每10个epoch打印一次输出；
        print('Epoch %d loss: %.3f' % (epoch + 1, running_loss / len(trainloader)))

Epoch 1 loss: 0.679
Epoch 11 loss: 0.675
Epoch 21 loss: 0.671
Epoch 31 loss: 0.667
Epoch 41 loss: 0.663
Epoch 51 loss: 0.659
Epoch 61 loss: 0.655
Epoch 71 loss: 0.651
Epoch 81 loss: 0.648
Epoch 91 loss: 0.644
